In [ ]:
!pip install sentence-transformers openai pinecone-client langchain_openai pinecone langchain-pinecone langchain_huggingface

In [ ]:
%pip install -qU pypdf langchain_community

In [ ]:
pip install -qU langchain-mistralai

In [ ]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = "your-api-key"

from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-large-latest")

PINECONE_API_KEY = "your-api-key"
PINECONE_ENV = "us-east-1"
INDEX_NAME = "legal-db"
NAMESPACE = "rag-legal"
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

··········


In [ ]:
# import os
# os.environ["USE_TF"] = "0"

In [ ]:
import os
from langchain_community.vectorstores import Pinecone as LangchainPinecone
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_pinecone import Pinecone, PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

In [ ]:
pc = Pinecone(api_key=PINECONE_API_KEY)
raw_index = pc.Index(INDEX_NAME)

In [ ]:
# !pip install --upgrade --force-reinstall tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.

In [ ]:
import tensorflow as tf

# Reset TensorFlow's default graph and clear the session
tf.compat.v1.reset_default_graph()
tf.compat.v1.keras.backend.clear_session()

In [ ]:
embeddings_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectorstore = PineconeVectorStore.from_existing_index(
    index_name=INDEX_NAME,
    embedding=embeddings_model,
    namespace=NAMESPACE
)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
prompt_template = ChatPromptTemplate.from_template("""
You are a trained bot to guide people about Indian Law. You will answer user's query with your knowledge and the context provided.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
Do not say thank you and tell you are an AI Assistant and be open about everything.
Use the following pieces of context to answer the users question.
Context: {context}
Question: {question}
Only return the helpful answer below and nothing else.
Helpful answer:
""")

In [ ]:
# --- RAG CHAIN ---
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)


In [ ]:
# --- MAIN CHAT LOOP ---
if __name__ == "__main__":
    print("🧠 LangChain Legal RAG Chatbot (type 'exit' to quit)")
    while True:
        query = input("\n🧑 You: ")
        if query.lower() in ["exit", "quit"]:
            break
        result = qa_chain.invoke({"query": query})
        print(f"\n🤖 Bot: {result}")

🧠 LangChain Legal RAG Chatbot (type 'exit' to quit)

🧑 You: what is the punishment for dowry in India



🤖 Bot: {'query': 'what is the punishment for dowry in India', 'result': 'The punishment for dowry demands and related offenses in India is outlined in the Dowry Prohibition Act, 1961. The penalties include:\n\n1. **Imprisonment**: A minimum of 5 years, which may extend to life imprisonment.\n2. **Fine**: A fine which shall not be less than Rs. 15,000 or the amount of the dowry, whichever is more.'}

🧑 You: can you explain it a bit more



🤖 Bot: {'query': 'can you explain it a bit more', 'result': "I'm afraid I can't provide more information because the context you've given is empty. Please provide some text or details so I can assist you better."}

🧑 You: tell me about the consumer laws in Inda



🤖 Bot: {'query': 'tell me about the consumer laws in Inda', 'result': "I'm unable to provide information on consumer laws in India as the context provided does not contain any relevant details."}

🧑 You: consumer Laws in India



🤖 Bot: {'query': 'consumer Laws in India', 'result': 'In India, consumer laws are primarily governed by the Consumer Protection Act, 2019, which replaced the earlier 1986 Act. Key provisions include:\n\n1. **Consumer Rights**: The Act recognizes six consumer rights—the right to be protected, the right to be informed, the right to choose, the right to be heard, the right to seek redressal, and the right to consumer education.\n\n2. **Central Consumer Protection Authority (CCPA)**: Established to regulate matters related to violation of consumer rights, unfair trade practices, and misleading advertisements.\n\n3. **Consumer Disputes Redressal Commissions**: District, State, and National Commissions handle consumer grievances.\n\n4. **Product Liability**: Introduces the concept of product liability, making manufacturers, product service providers, and product sellers liable for any harm caused to consumers due to defective products or deficient services.\n\n5. **E-Commerce**: Specific pr


🤖 Bot: {'query': 'tell me about the criminal laws in India. when does a person get death sentence', 'result': 'In India, criminal laws are primarily governed by the Indian Penal Code (IPC), 1860, and the Code of Criminal Procedure (CrPC), 1973. The death sentence, also known as capital punishment, is the most severe penalty provided by the Indian legal system. It is typically reserved for the "rarest of rare" cases, as outlined by the Supreme Court of India.\n\nCircumstances that may warrant the death sentence include:\n\n1. **Murder**: Particularly heinous or brutal murders, including those involving multiple victims, premeditation, or aggravating factors.\n2. **Terrorism**: Acts of terrorism that result in loss of life or significant damage to property.\n3. **Rape and Murder**: Cases where the victim is raped and then murdered.\n4. **Kidnapping for Ransom**: Especially if it results in the death of the victim.\n5. **Waging War Against the State**: Acts of treason or rebellion agains


🤖 Bot: {'query': 'what are the laws related to firearm ownership in India', 'result': 'Based on the context provided, there is no information available regarding the laws related to firearm ownership in India. Therefore, I cannot provide an answer to your question.'}

🧑 You: who can have guns in India



🤖 Bot: {'query': 'who can have guns in India', 'result': 'In India, the possession of guns is strictly regulated by the Arms Act, 1959, and the Arms Rules, 2016. Generally, only licensed individuals can possess firearms. Licenses are issued by the authorities and are typically granted for specific purposes such as self-defense, sports, or hunting. The criteria for obtaining a license include background checks, proof of need, and sometimes training. Certain categories of individuals, such as those with criminal records or mental health issues, are generally prohibited from obtaining a license.'}

🧑 You: exit
